In [1]:
from cosapp.base import System, Port, BaseConnector
import numpy as np
from cosapp.drivers import RungeKutta
import plotly.graph_objs as go
from cosapp.recorders import DataFrameRecorder
import plotly.express as px
import matplotlib.pyplot as plt

In [2]:
def RotMat2D(theta):
    
    A = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
            
    return A

def RotMat3D(ang):
    
    #ang[0] = phi, ang[1] = theta, ang[2] = psi
    
    l1 = np.array([np.cos(ang[2])*np.cos(ang[1]), -np.sin(ang[2])*np.cos(ang[0]) + np.cos(ang[2])*np.sin(ang[1])*np.sin(ang[0]),
                    np.sin(ang[2])*np.sin(ang[0]) + np.cos(ang[2])*np.sin(ang[1])*np.cos(ang[0])])
    l2 = np.array([np.sin(ang[2])*np.cos(ang[1]),  np.cos(ang[2])*np.cos(ang[0]) + np.sin(ang[2])*np.sin(ang[1])*np.sin(ang[0]),
                   -np.cos(ang[2])*np.sin(ang[0]) + np.sin(ang[2])*np.sin(ang[1])*np.cos(ang[0])])
    l3 = np.array([-np.sin(ang[1]), np.cos(ang[1])*np.sin(ang[0]), np.cos(ang[1])*np.cos(ang[0])])
    
    A = np.round(np.array([l1,l2,l3]), 5)
    #print(A)
    
    return A

In [3]:
class RotPort2(Port):
    """Position Port """
    def setup(self):
        self.add_variable('val', np.zeros(2))
        self.add_variable('ang', 0.)
        

    class Connector(BaseConnector):
        """Custom connector for `PosPort` objects
        """
        def __init__(self, name: str, sink: Port, source: Port, *args, **kwargs):
            super().__init__(name, sink, source)

        def transfer(self) -> None:
            sink = self.sink
            source = self.source
            
            sink.val = np.matmul(RotMat2D(-source.ang), source.val)
            
            #if sink.owner.parent == source.owner:
                #sink.pos = np.matmul(RotMat2D(source.ang), source.pos)

            #if sink.owner == source.owner.parent:
                #sink.pos = np.matmul(RotMat2D(-source.ang), source.pos)
                
class RotPort3(Port):
    """Position Port """
    def setup(self):
        self.add_variable('val', np.zeros(3))
        self.add_variable('ang', np.zeros(3))
        

    class Connector(BaseConnector):
        """Custom connector for `PosPort` objects
        """
        def __init__(self, name: str, sink: Port, source: Port, *args, **kwargs):
            super().__init__(name, sink, source)

        def transfer(self) -> None:
            sink = self.sink
            source = self.source
            
            sink.val = np.matmul(RotMat3D(-source.ang), source.val)
            
            #if sink.owner.parent == source.owner:
                #sink.pos = np.matmul(RotMat2D(source.ang), source.pos)

            #if sink.owner == source.owner.parent:
                #sink.pos = np.matmul(RotMat2D(-source.ang), source.pos)

In [4]:
class Rocket(System):
    
    def setup(self):
        
        #Parent-Child connections
        self.add_child(Dynamics('Dyn'), pulling = ['a', 'aa', 'av', 'm'])
        self.add_child(Thrust('Thrust'))
        self.add_child(Mass('Mass'), pulling = ['dmdt', 'm0'])
        self.add_child(Inertia('Inertia'), pulling = ['m', 'm0'])
        self.add_child(Gravity('Grav'), pulling = ['ar'])
        self.add_child(Atmosphere('Atmo'))
        self.add_child(Trajectory('Traj'), pulling = ['v_out'])
        
        #Child-Child connections
        self.connect(self.Dyn, self.Thrust, ['Fp', 'Mp'])
        self.connect(self.Dyn, self.Inertia, ['I'])
        self.connect(self.Dyn, self.Grav, ['g'])
        self.connect(self.Traj, self.Grav, ['r'])
        self.connect(self.Traj, self.Atmo, ['r'])
        self.connect(self.Thrust, self.Atmo, ['P'])
        
        #Kinematic variables
        self.add_transient('v', der = 'a.val')
        self.add_transient('av', der = 'aa.val')
        self.add_transient('ar', der = 'av')
        self.add_transient('m', der = 'dmdt')
        
        #Execution order
        self.exec_order = ['Mass', 'Inertia', 'Grav', 'Atmo', 'Thrust', 'Dyn', 'Traj']
        
    def compute(self):
        
        self.v_out.val = self.v
        self.v_out.ang = self.ar
    
    

In [5]:
class Dynamics(System):
    
    def setup(self):
        
        #Rocket inputs
        self.add_inward('v', np.zeros(3), desc = "Rocket's Velocity")
        self.add_inward('av', np.zeros(3), desc = "Rocket's Angular Velocity")
        
        #Thrust inputs
        self.add_input(RotPort3, 'Fp')
        self.add_input(RotPort3, 'Mp')
        
        #Mass inputs
        self.add_inward('m', 100.)
        
        #Inertia inputs
        self.add_inward('I', np.ones(3))
        
        #Gravity inputs
        self.add_input(RotPort3, 'g')
        
        #Thrust outputs
        self.add_output(RotPort3, 'a')
        self.add_output(RotPort3, 'aa')
        
        
    def compute(self):
        
        self.a.val[0] = (self.Fp.val[0])/self.m + self.g.val[0] + self.av[2]*self.v[1] - self.av[1]*self.v[2]
        self.a.val[1] = (self.Fp.val[1])/self.m + self.g.val[1] + self.av[0]*self.v[2] - self.av[2]*self.v[0]
        self.a.val[2] = (self.Fp.val[2])/self.m + self.g.val[2] + self.av[1]*self.v[0] - self.av[0]*self.v[1]
        
        self.aa.val[0] = (self.Mp.val[0] + (self.I[1] - self.I[2])*self.av[1]*self.av[2])/self.I[0]
        self.aa.val[1] = (self.Mp.val[1] + (self.I[2] - self.I[0])*self.av[2]*self.av[0])/self.I[1]
        self.aa.val[2] = (self.Mp.val[2] + (self.I[0] - self.I[1])*self.av[0]*self.av[1])/self.I[2]
        

In [6]:
class Thrust(System):
    
    def setup(self):
        
        #System Constants
        self.add_inward('g0', 9.81, desc = "Gravity on Earth's Surface", unit = 'm/s**2')
        
        #Rocket inputs
        self.add_inward('qp', 10., desc = "Engine's Propulsive Debit", unit = 'kg/s')
        self.add_inward('isp', 100., desc = "Specific Impulsion in Vacuum", unit = 's')
        self.add_inward('gc', 1., desc = "GC Distance")
        self.add_inward('S_ref', 0.01, desc = "Surface of Reference")
        
        #Atmosphere inputs
        self.add_inward('P', 101325., desc = "Atmospheric Pression at Rocket's Height")
        
        #Thrust outputs
        self.add_output(RotPort3, 'Fp')
        self.add_output(RotPort3, 'Mp')
        
        
    def compute(self):
        
        self.Fp.val[0] = self.qp*self.g0*self.isp - self.P*self.S_ref
        self.Fp.val[1] = 0.
        self.Fp.val[2] = 0.
        
        self.Mp.val[0] = 0.
        self.Mp.val[1] = 0.
        self.Mp.val[2] = 0.
        
        

In [7]:
class Mass(System):
    
    def setup(self):
        
        #Rocket inputs
        self.add_inward('m0', 100., desc = "Rocket's Initial Mass")
        self.add_inward('qp', 8., desc = "Engine's Propulsive Debt")
        self.add_inward('qi', 2., desc = "Engine's Inertial Debt")
        self.add_inward('m', 1., desc = "Rocket Mass")
        
        #Mass outputs
        self.add_outward('dmdt', 1., desc = "Rocket Mass' Rate of Change")
        
    def compute(self):
        
        self.dmdt = -(self.qp + self.qi)
    

In [8]:
class Inertia(System):
    
    def setup(self):
        
        #Rocket inputs
        self.add_inward('m0', 100., desc = "Rocket's Initial Mass")
        self.add_inward('m', 100., desc = "Rocket Mass")
        self.add_inward('I0', np.array([10., 100., 100.]), desc = "Rocket's Initial Moments of Inertia")
        
        #Inertia outputs
        self.add_outward('I', np.ones(3), desc = "Rocket's Moments of Inertia")
        
    def compute(self):
        
        self.I = self.I0*self.m/self.m0
    

In [9]:
class Gravity(System):
    
    def setup(self):
        
        #System Constants
        self.add_inward('G', 6.6743*10**(-11), desc = "Gravitational Constant")
        self.add_inward('M', 5.972*10**24, desc = "Earth's Mass")
        self.add_inward('R', 6.371*10**6, desc = "Earth's Radius")
        
        #Rocket inputs
        self.add_inward('ar', np.zeros(3), desc = "Rocket's Angular Position")
        
        #Trajectory inputs
        self.add_inward('r', np.zeros(3), desc = "Rocket's Position")
        
        #Gravity outputs
        self.add_output(RotPort3, 'g')
        
    def compute(self):
        
        self.g.val = np.array([0., 0., -self.G*self.M/(self.R + self.r[-1])**2])
        self.g.ang = -self.ar
        
        
        

In [10]:
class Atmosphere(System):
    
    def setup(self):
        
        #System constants
        self.add_inward('rho0', 1.225, desc = "Air Density at Sea Level")
        self.add_inward('P0', 101325., desc = "Atmospheric Pressure at Sea Level")
        self.add_inward('V', 0., desc = "Wind speed")
        self.add_inward('eps', 0., desc = "Wind Direction")
        
        #Rocket inputs
        self.add_inward('ar', np.zeros(3), desc = "Rocket's Angular Position")
        
        #Trajectory inputs
        self.add_inward('r', np.zeros(3), desc = "Rocket's Position")
        
        #Atmosphere outputs
        self.add_outward('rho', 1.225, desc = "Air Density at Rocket's Height")
        self.add_outward('P', 101325., desc = "Atmospheric Pressure at Rocket's Height")
        self.add_output(RotPort3, 'V_wind')
        
    def compute(self):
        
        self.rho = self.rho0 * (20000. - self.r[-1]) / (20000. + self.r[-1])
        self.P = self.P0 * (20000. - self.r[-1]) / (20000. + self.r[-1])
        self.V_wind.val = self.V
        self.V_wind.ang = -(self.ar + np.array([0., 0., self.eps]))
    

In [11]:
class Aerodynamics(System):
    
    
    
    

IndentationError: expected an indented block (214757741.py, line 5)

In [13]:
class Trajectory(System):
    
    def setup(self):
        
        #Rocket inputs
        self.add_input(RotPort3, 'v_out')
        self.add_transient('r', der = 'v_out.val')
        
        

In [14]:
#Main function

#Time-step
dt = 0.01

#Add RungeKutta driver
rocket = Rocket('rocket')
driver = rocket.add_driver(RungeKutta(order=4, dt=dt))
driver.time_interval = (0, 9)

# Add a recorder to capture time evolution in a dataframe
driver.add_recorder(
    DataFrameRecorder(includes=['Traj.r', 'v', 'a.val', 'ar', 'av', 'aa.val']),
    period=1,
)

#Initial conditions and constants
driver.set_scenario(
    init = {
        'v' : np.zeros(3),
        'ar' : np.array([0., np.pi/2, 0.]),
        'av' : 0.*np.array([np.pi/20, np.pi/20, 0.])
    })


rocket.run_drivers()

In [19]:
rocket

rocket - Rocket

In [15]:
data = driver.recorder.export_data()
data = data.drop(['Section', 'Status', 'Error code'], axis=1)
time = np.asarray(data['time'])
traj = np.asarray(data['Traj.r'].tolist())
velo = np.asarray(data['v'].tolist())
acel = np.asarray(data['a.val'].tolist())
atraj = np.asarray(data['ar'].tolist())
avelo = np.asarray(data['av'].tolist())
aacel = np.asarray(data['aa.val'].tolist())

In [16]:
traj

array([[   0.        ,    0.        ,    0.        ],
       [   0.        ,    0.        ,   40.62151012],
       [   0.        ,    0.        ,  169.4226479 ],
       [   0.        ,    0.        ,  398.88001963],
       [   0.        ,    0.        ,  745.20774882],
       [   0.        ,    0.        , 1230.26996893],
       [   0.        ,    0.        , 1885.06322534],
       [   0.        ,    0.        , 2756.81935115],
       [   0.        ,    0.        , 3926.21692966],
       [   0.        ,    0.        , 5564.04355413]])

In [17]:
velo

array([[   0.        ,    0.        ,    0.        ],
       [  82.87775655,    0.        ,    0.        ],
       [ 176.78572985,    0.        ,    0.        ],
       [ 284.80114527,    0.        ,    0.        ],
       [ 411.44433188,    0.        ,    0.        ],
       [ 563.73945321,    0.        ,    0.        ],
       [ 753.47519433,    0.        ,    0.        ],
       [1002.78473644,    0.        ,    0.        ],
       [1361.43008102,    0.        ,    0.        ],
       [1988.45944084,    0.        ,    0.        ]])

In [18]:
acel

array([[ 78.14752657,   0.        ,   0.        ],
       [ 87.96745881,   0.        ,   0.        ],
       [100.3527057 ,   0.        ,   0.        ],
       [116.41520199,   0.        ,   0.        ],
       [138.00808637,   0.        ,   0.        ],
       [168.46748539,   0.        ,   0.        ],
       [214.46838404,   0.        ,   0.        ],
       [291.59669946,   0.        ,   0.        ],
       [446.65673253,   0.        ,   0.        ],
       [913.97916261,   0.        ,   0.        ]])

In [ ]:
atraj

In [ ]:
avelo

In [ ]:
aacel

In [ ]:
#Animation - Center of Mass Movement

fig=go.Figure(go.Scatter(x=[traj[0,0]], y=[traj[0,2]], mode="markers", marker = dict(color="red", size=10), name='Testing Points'))
fig.update_layout(title='Center of Gravity Movement',
                  title_x=0.5,
                  width=600, height=600, 
                  xaxis_title='Ground Level', 
                  yaxis_title='Height',
                  yaxis_range=(0,4000),
                  xaxis_range=(0,4000), #you generate y-values for i =0, ...99, 
                                      #that are assigned, by default, to x-values 0, 1, ..., 99
                  
                  updatemenus=[dict(buttons = [dict(
                                               args = [None, {"frame": {"duration": 10000*dt, 
                                                                        "redraw": True},
                                                              "fromcurrent": True, 
                                                              "transition": {"duration": 10}}],
                                               label = "Play",
                                               method = "animate")],
                                type='buttons',
                                showactive=False,
                                y=1,
                                x=1.12,
                                xanchor='right',
                                yanchor='top')])


                                          
                    
frames= [go.Frame(data=[go.Scatter(x=[traj[i,0]],y=[traj[i,2]])]) for i in range(len(traj[:,0]))]
fig.update(frames=frames)

fig.show()

In [ ]:
plt.scatter(traj[:,0], traj[:,2])

In [ ]:
traj[:,2]

In [ ]:
traj[:,0]